# NLP training example
In this example, we'll train an NLP model for sentiment analysis of tweets using spaCy.

First we download spaCy language libraries.

In [1]:
!python -m spacy download en_core_web_sm

You should consider upgrading via the 'pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


And import the boilerplate code.

In [2]:
from __future__ import unicode_literals, print_function

import boto3
import json
import numpy as np
import pandas as pd
import spacy

## Data prep

Download the dataset from S3

In [3]:
S3_BUCKET = "verta-strata"
S3_KEY = "english-tweets.csv"
FILENAME = S3_KEY

boto3.client('s3').download_file(S3_BUCKET, S3_KEY, FILENAME)

Load data using our library.

In [4]:
import utils

data = pd.read_csv(FILENAME).sample(frac=1).reset_index(drop=True)

data.head()

,text,sentiment
0,I miss you too.. But at this time we can't ch...,0
1,with tanner,1
2,yess..Leet World Episode 10 is out.. Player ft...,1
3,@SolOtis I just wish I had a partner in this. ...,1
4,@novemberborn whishing I could select tabs in ...,0


## Set up ModelDB
ModelDB organizes our work, and enables us to log and version metadata.

In [5]:
from verta import Client

client = Client('https://dev.verta.ai')
client.set_project('Tweet Classification')
client.set_experiment('SpaCy')
run = client.set_experiment_run()

set email from environment
set developer key from environment
connection successfully established
set existing Project: Tweet Classification from personal workspace
set existing Experiment: SpaCy
created new ExperimentRun: Run 992821584404891576972


We'll first record our code, configuration, dataset, and environment versions to a ModelDB repository.

In [6]:
repo = client.set_repository('Verta Strata')
commit = repo.get_commit(branch='master')

set existing Repository: Verta Strata from personal workspace


In [7]:
from verta.code import Notebook
from verta.configuration import Hyperparameters
from verta.dataset import S3
from verta.environment import Python

code_ver = Notebook()
config_ver = Hyperparameters({'n_iter': 20})
dataset_ver = S3("s3://{}/{}".format(S3_BUCKET, S3_KEY))
env_ver = Python()

commit.update("notebooks/tweet-analysis", code_ver)
commit.update("config/hyperparams", config_ver)
commit.update("data/tweets", dataset_ver)
commit.update("env/python", env_ver)
commit.save("Deployment-ready sentiment analysis")

commit

<IPython.core.display.Javascript object>

(Branch: master)
Commit a667ec6a712ca1f751722843484977da34eb0e12a8b3d35c0d412b0821af0929 containing:
config/hyperparams (Blob)
data/tweets (Blob)
env/python (Blob)
notebooks/tweet-analysis (Blob)

## Train the model
We'll use a pre-trained model from spaCy and fine tune it in our new dataset.

In [8]:
nlp = spacy.load('en_core_web_sm')

Update the model with the current data using our library.

In [9]:
import training

training.train(nlp, data, n_iter=20)

Using 16000 examples (12800 training, 3200 evaluation)
Training the model...
LOSS 	  P  	  R  	  F  
15.801	0.724	0.728	0.726
0.355	0.743	0.730	0.737
0.104	0.748	0.729	0.738
0.088	0.756	0.736	0.746
0.075	0.748	0.741	0.744
0.063	0.743	0.741	0.742
0.050	0.743	0.745	0.744
0.042	0.743	0.742	0.743
0.035	0.746	0.754	0.750
0.030	0.746	0.750	0.748
0.026	0.741	0.751	0.746
0.023	0.736	0.743	0.739
0.021	0.736	0.741	0.739
0.020	0.734	0.741	0.737
0.017	0.736	0.737	0.736
0.015	0.736	0.732	0.734
0.016	0.742	0.734	0.738
0.013	0.746	0.729	0.738
0.012	0.746	0.729	0.737
0.012	0.745	0.730	0.737


## Save and version the model
We log the model itself as an artifact to ModelDB.

In [ ]:
run.log_model(nlp)

upload complete (custom_modules.zip)


And finally, link the commit to our Experiment Run.

In [ ]:
run.log_commit(
    commit,
    {
        'notebook': "notebooks/tweet-analysis",
        'hyperparameters': "config/hyperparams",
        'training_data': "data/tweets",
        'python_env': "env/python",
    },
)

## Deployment

Great! Now you have a model that you can use to run predictions against. Follow the next step of this tutorial to see how to do it.